In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3

In [2]:
import numpy as np
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import zipfile

# zip_ref = zipfile.ZipFile("/content/drive/MyDrive/ML-FitFood/Data.zip", 'r')
# zip_ref.extractall("Data/")
# zip_ref.close()

train_data_dir = ('Data/Dataset/Train')
test_data_dir = ('Data/Dataset/Test')
valid_data_dir = ('Data/Dataset/Val')

img_width, img_height = 224, 224
batch_size = 32
input_shape = (img_width, img_height, 3)

train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-label classification
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 10080 images belonging to 42 classes.
Found 1260 images belonging to 42 classes.
Found 1260 images belonging to 42 classes.


In [ ]:
from tensorflow.keras.applications import InceptionV3
# Load the InceptionV3 model pre-trained on ImageNet
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=10  # You can start with a small number of epochs and increase as needed
)

# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[:249]:
    layer.trainable = False
for layer in base_model.layers[249:]:
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Continue training the model
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=100  # Additional epochs for fine-tuning
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test accuracy: {test_accuracy:.2f}")

Epoch 1/10
21/21 [==============================] - 8s 193ms/step - loss: 0.3566 - accuracy: 0.0905 - val_loss: 0.1801 - val_accuracy: 0.3063
Epoch 2/10
21/21 [==============================] - 2s 97ms/step - loss: 0.2360 - accuracy: 0.2090 - val_loss: 0.1532 - val_accuracy: 0.5250
Epoch 3/10
21/21 [==============================] - 2s 94ms/step - loss: 0.1968 - accuracy: 0.2980 - val_loss: 0.1363 - val_accuracy: 0.6062
Epoch 4/10
21/21 [==============================] - 2s 107ms/step - loss: 0.1695 - accuracy: 0.3729 - val_loss: 0.1297 - val_accuracy: 0.6313
Epoch 5/10
21/21 [==============================] - 2s 105ms/step - loss: 0.1474 - accuracy: 0.4727 - val_loss: 0.1093 - val_accuracy: 0.7125
Epoch 6/10
21/21 [==============================] - 2s 104ms/step - loss: 0.1359 - accuracy: 0.5429 - val_loss: 0.1107 - val_accuracy: 0.6875
Epoch 7/10
21/21 [==============================] - 2s 110ms/step - loss: 0.1358 - accuracy: 0.5507 - val_loss: 0.1147 - val_accuracy: 0.6875
Epoch 8/

In [7]:
# Define input shape
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)  # Adjust this if your input shape is different

# Define num_classes based on train_generator
num_classes = train_generator.num_classes

# Build the model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten layer
model.add(Flatten())

# Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))  # Use 'sigmoid' for multi-label classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [8]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=50
)

test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test accuracy: {test_accuracy:.2f}")


Epoch 1/50
 61/315 [====>.........................] - ETA: 1:23 - loss: 0.2271 - accuracy: 0.0292

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


315/315 [==============================] - 131s 386ms/step - loss: 0.1431 - accuracy: 0.0309 - val_loss: 0.1101 - val_accuracy: 0.0673
Epoch 2/50
315/315 [==============================] - 112s 357ms/step - loss: 0.1141 - accuracy: 0.0616 - val_loss: 0.0999 - val_accuracy: 0.0994
Epoch 3/50
315/315 [==============================] - 116s 367ms/step - loss: 0.1047 - accuracy: 0.1064 - val_loss: 0.0912 - val_accuracy: 0.1715
Epoch 4/50
315/315 [==============================] - 113s 357ms/step - loss: 0.0987 - accuracy: 0.1417 - val_loss: 0.0884 - val_accuracy: 0.2019
Epoch 5/50
315/315 [==============================] - 114s 361ms/step - loss: 0.0943 - accuracy: 0.1718 - val_loss: 0.0860 - val_accuracy: 0.2171
Epoch 6/50
315/315 [==============================] - 114s 362ms/step - loss: 0.0901 - accuracy: 0.2102 - val_loss: 0.0834 - val_accuracy: 0.2821
Epoch 7/50
315/315 [==============================] - 123s 389ms/step - loss: 0.0851 - accuracy: 0.2520 - val_loss: 0.0811 - val_accura

In [ ]:
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Conv2D(128, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(train_generator.num_classes, activation='sigmoid')  # Use sigmoid for multi-label classification
# ])

# model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
model_path = '/content/drive/My Drive/model.h5'
model.save(model_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# epochs = 100
# model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // batch_size,
#     epochs=epochs,
#     validation_data=valid_generator,
#     validation_steps=valid_generator.samples // batch_size
# )

# # Save the model
model.save('multi_label_modelw2.h5')


In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Accuracy: {test_acc}, Test Loss: {test_loss}')


In [33]:
# Load the model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
model = load_model('multi_label_modelw2.h5')

# Function to preprocess new image
def preprocess_image(img_path, target_size):
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

# img_path = 'naskunxsate.jpg'
img_path = 'nasxayam.jpg'
# img_path = 'ayamxtest.jpg'
# img_path = 'rawonxtest.jpg'
processed_image = preprocess_image(img_path, target_size=(224, 224))
predictions = model.predict(processed_image)
threshold = 0.1
predicted_labels = (predictions > threshold).astype(int)
label_map = train_generator.class_indices
label_map = dict((v,k) for k,v in label_map.items())
predicted_classes = [label_map[i] for i in range(len(predicted_labels[0])) if predicted_labels[0][i] == 1]
print(f'Predicted labels for the image: {predicted_classes}')


1/1 [==============================] - 0s 77ms/step
Predicted labels for the image: ['Ayam_Crispy', 'Nasi_Goreng']


In [13]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_model_path = '/content/drive/My Drive/model.tflite'

In [18]:
from google.colab import files

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


In [19]:
tflite_model_path = 'model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"Model TensorFlow Lite disimpan di: {tflite_model_path}")
files.download(tflite_model_path)

Model TensorFlow Lite disimpan di: model.tflite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>